In [60]:
import pandas as pd
import numpy as np
from math import exp

In [3]:
path = "data/homework_02-data.csv"
df = pd.read_csv(path)

In [5]:
df.head()

,subject,x,f,g
0,1,0.000,-1.171990,-0.342031
1,1,0.001,-1.082194,-0.327248
2,1,0.002,-0.952844,-0.297205
3,1,0.003,-0.793758,-0.263112
4,1,0.004,-0.614753,-0.226987


In [63]:
def scale(x: float, l: float, h: float):
    """Scale value between the minimum, l, and maximum, h.
    Pass in l and h so we don't need to recalculate min and max so often.
    Inspired by: https://stackoverflow.com/questions/18380419/normalization-to-bring-in-the-range-of-0-1
    """
    return (x-l)/(h-l)

def warp(x: int, B: tuple):
    """Warp function that partially defines f and g."""
    b1, b2, b3 = B
    return (exp(b1)*x + exp(b2)*x**exp(b3))/exp(b1)+exp(b2)

In [64]:
def preprocess(subject: int, data: pd.DataFrame):
    """Preprocesses data by selecting subject i from the data, scaling f_i and g_i to [0,1], and returning a
    function to evaluate f(x) and g(x_tilde).
    Returns a function that will evaluate f and g at many x for a given tuple of betas.
    """
    #select subject data
    subject_df = df[df["subject"] == subject]
    
    #expand x as int
    x_int = subject_df["x"].map(lambda x: int(x*1000))
    
    #scale function values to [0,1]
    F = subject_df["f"]
    l = np.min(F)
    h = np.max(F)
    f_scaled = F.map(lambda f: scale(f, l, h))

    #now scale g
    G = subject_df["g"]
    l = np.min(G)
    h = np.max(G)
    g_scaled = G.map(lambda g: scale(g, l, h))
    
    #define and return function which evaluates f(x) and g(x_tilde)
    #TODO: what's x_tilde?
    #hm... are we just looking something up?
    
    #you need to evaluate g to evaluate f... right?
    def beta_cruncher(betas: tuple):
        G = [warp(g_scaled[x],betas) for x in x_int] #this is evaluating w(x,B)
        #what about f? there are some unresolved values (gamma and eta)
        
        return G #returns evaluated F and G
        
    return beta_cruncher

In [65]:
b = preprocess(1,df)

In [67]:
b((0.1,0.2,0.3))

[1.9936609192893038,
 2.001576952968802,
 2.0177028912382853,
 2.0360664208621055,
 2.0555967528148327,
 2.080112845838649,
 2.1108665310427415,
 2.1328178295045035,
 2.1628923886772116,
 2.1893285433330534,
 2.210176084832796,
 2.2343356448490606,
 2.249250998216983,
 2.261618852446387,
 2.260294424174476,
 2.2568808212701272,
 2.248268242469212,
 2.245009143281967,
 2.2331333765060464,
 2.218990674757146,
 2.203830333544401,
 2.1931746441082636,
 2.1752099623441437,
 2.158360317585195,
 2.1462439299381493,
 2.138557528347773,
 2.128746749728361,
 2.1220414316209184,
 2.1186468042297717,
 2.1186531776171575,
 2.1191000395345503,
 2.121243433865674,
 2.1300788965643194,
 2.1399933700388942,
 2.1460851467282076,
 2.156879699990519,
 2.1679278945981806,
 2.1827396674984154,
 2.1939516322226478,
 2.2067140630676065,
 2.2255963823546967,
 2.2453209273538266,
 2.2612080696390047,
 2.273056876485616,
 2.285164904293346,
 2.292381277333042,
 2.292104016850694,
 2.2931065904662202,
 2.29451009

In [61]:
def warp(x: int, B: tuple):
    """Warp function that partially defines f and g."""
    b1, b2, b3 = B
    return (exp(b1)*x + exp(b2)*x**exp(b3))/exp(b1)+exp(b2)

In [62]:
warp(0.1, (0.2,0.3,0.4))

1.4854714952190795

In [51]:
def foo():
    def fark(a: int):
        return a+5
    return fark

In [57]:
a = foo()
a(3)

8

In [44]:
preprocess(1, df)